This file runs each step of pipeline seperately as a debug tool

In [1]:
from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from ml_metadata.proto import metadata_store_pb2
from pipeline.e2e_pipeline.pipeline import create_pipeline

In [2]:
from pipeline.config import pipe_config

In [3]:
connection_config = metadata_store_pb2.ConnectionConfig()
connection_config.sqlite.filename_uri = pipe_config.METADATA_PATH
connection_config.sqlite.connection_mode = 3
# Or
metadata_connection_config = tfx.orchestration.metadata.sqlite_metadata_connection_config(
    pipe_config.METADATA_PATH)
    
context = InteractiveContext(pipeline_root=pipe_config.PIPELINE_ROOT, metadata_connection_config=connection_config)


In [4]:
pipeline = create_pipeline(
        pipeline_name=pipe_config.PIPELINE_NAME,
        pipeline_root=pipe_config.PIPELINE_ROOT,
        data_root=pipe_config.DATA_ROOT,
        metadata_path=pipe_config.METADATA_PATH,
        schema_path=pipe_config.SAVED_SCHEMA_PATH,
        trainer_module_file=pipe_config.TRAINER_MODULE_PATH,
        serving_model_dir=pipe_config.SERVING_MODEL_DIR,
    )

In [5]:
pipeline.components

[CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7feb760c3b50>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7feb760c3be0>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': Channel(
     type_name: Examples
     artifacts: []
     additional_properties: {}
     additional_custom_properties: {}
 )}),
 StatisticsGen(spec: <tfx.types.standard_component_specs.StatisticsGenSpec object at 0x7feb760b9e50>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7feb760b9f10>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: StatisticsGen, inputs: {'examples': Channel(
     type_name: Examples
     artifacts: []
     additional_properties: {}
     additional_custom_properties: {}
 )}, outputs: {'statistics': Channel(
     type_name: ExampleStatistics
     artifac

In [6]:
# get lattest trainer artifacts
from pipeline.schema_pipeline.utils import get_latest_artifacts
from tfx.orchestration.metadata import Metadata
from tfx.types import standard_component_specs

with Metadata(metadata_connection_config) as metadata_handler:
    # Find output artifacts from MLMD.
    trainer_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'Trainer')
    example_gen_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'CsvExampleGen')
    evaluator_outputs = get_latest_artifacts(metadata_handler, pipe_config.PIPELINE_NAME,
                                          'Evaluator')

In [7]:
# from tfx.types import channel_utils
# # run Resolver

# resolver = pipeline.components[1]
# resolver.inputs['model'] = channel_utils.as_channel(trainer_outputs['model'])
# resolver.inputs['model_blessing'] = channel_utils.as_channel(evaluator_outputs['blessing'])

# context.run(resolver)
# resolver.inputs

In [8]:
pipeline.components[5]

Evaluator(spec: <tfx.types.standard_component_specs.EvaluatorSpec object at 0x7feb761df0a0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7feb761df160>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Evaluator, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'model': Channel(
    type_name: Model
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'baseline_model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: , artifact_type: name: "Model"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [11]:
from tfx.types import channel_utils
# get latest evaluator artifact
evaluator = pipeline.components[5]
# convert trainer Model artifact to Channel
evaluator.inputs['model'] = channel_utils.as_channel(trainer_outputs['model'])
evaluator.inputs['examples'] = channel_utils.as_channel(example_gen_outputs['examples'])
evaluator.inputs['baseline_model'] = channel_utils.as_channel(evaluator_outputs['blessing'])

In [12]:
evaluator

Evaluator(spec: <tfx.types.standard_component_specs.EvaluatorSpec object at 0x7feb761df0a0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7feb761df160>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Evaluator, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 6
uri: "pipeline_output/penguin-e2e/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:14417,xor_checksum:1627907631,sum_checksum:1627907631"
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "penguin-e2e:2021-08-20T01:08:50.494035:CsvExampleGen:examples:0"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    int_value: 0
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.0.0"
  }
}
state: LIVE
create_time_since_epoch: 1629421732190
last_update_time_since_epoch: 1629421732190
, artifact_type: id: 6
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 5
type_id: 14
uri: "pipeline_output/penguin-e2e/Trainer/model/5"
custom_properties {
  key: "name"
  value {
    string_value: "penguin-e2e:2021-08-20T01:08:50.494035:Trainer:model:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.0.0"
  }
}
state: LIVE
create_time_since_epoch: 1629421744378
last_update_time_since_epoch: 1629421744378
, artifact_type: id: 14
name: "Model"
)]
    additional_properties: {}
    additional_custom_properties: {}
), 'baseline_model': Channel(
    type_name: ModelBlessing
    artifacts: [Artifact(artifact: id: 6
type_id: 16
uri: "pipeline_output/penguin-e2e/Evaluator/blessing/6"
custom_properties {
  key: "blessed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "current_model"
  value {
    string_value: "pipeline_output/penguin-e2e/Trainer/model/5"
  }
}
custom_properties {
  key: "current_model_id"
  value {
    int_value: 5
  }
}
custom_properties {
  key: "name"
  value {
    string_value: "penguin-e2e:2021-08-20T01:08:50.494035:Evaluator:blessing:0"
  }
}
custom_properties {
  key: "tfx_version"
  value {
    string_value: "1.0.0"
  }
}
state: LIVE
create_time_since_epoch: 1629421749079
last_update_time_since_epoch: 1629421749079
, artifact_type: id: 16
name: "ModelBlessing"
)]
    additional_properties: {}
    additional_custom_properties: {}
)}, outputs: {'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [1]:
# context = InteractiveContext(pipeline_root=pipe_config.PIPELINE_ROOT, metadata_connection_config=connection_config)
# context.run(evaluator)